In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import random
from datetime import datetime, timedelta

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("BitcoinDataConsumer") \
    .enableHiveSupport() \
    .getOrCreate()

25/04/28 13:41:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
!pip install confluent_kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.2 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install httpx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [6]:
!pip install authlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 5.0 MB/s eta 0:00:00a 0:00:01


In [ ]:
import threading
from confluent_kafka import DeserializingConsumer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer
from confluent_kafka.serialization import StringDeserializer
import json



# Define Kafka configuration
kafka_config = {
    'bootstrap.servers': 'Cluster-server',
    'sasl.mechanisms': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': 'Cluster-username',
    'sasl.password': 'Cluster-password',
    'group.id': '5'
}

# Create a Schema Registry client
schema_registry_client = SchemaRegistryClient({
  'url': 'Schema-registry-endpoint',
  'basic.auth.user.info': '{}:{}'.format('Schema-registry-username', 'Schema-registry-password')
})

# Fetch the latest Avro schema for the value
subject_name = 'bitcoin_stream-value'
schema_str = schema_registry_client.get_latest_version(subject_name).schema.schema_str

# Create Avro Deserializer for the value
key_deserializer = StringDeserializer('utf_8')
avro_deserializer = AvroDeserializer(schema_registry_client, schema_str)

# Define the DeserializingConsumer
consumer = DeserializingConsumer({
    'bootstrap.servers': kafka_config['bootstrap.servers'],
    'security.protocol': kafka_config['security.protocol'],
    'sasl.mechanisms': kafka_config['sasl.mechanisms'],
    'sasl.username': kafka_config['sasl.username'],
    'sasl.password': kafka_config['sasl.password'],
    'key.deserializer': key_deserializer,
    'value.deserializer': avro_deserializer,
    'group.id' : kafka_config['group.id'],
    'auto.offset.reset': 'latest',
    'enable.auto.commit': True,
    'auto.commit.interval.ms': 5000 # Commit every 5000 ms, i.e., every 5 seconds
})

# Subscribe to the 'retail_data' topic
consumer.subscribe(['bitcoin_stream'])






from datetime import datetime
import time

buffer = []
current_minute = None

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print('Consumer error: {}'.format(msg.error()))
            continue

        msg_data = msg.value()
        timestamp_str = msg_data.get("timestamp")
        price = float(msg_data.get("price"))
        dt = datetime.fromisoformat(timestamp_str)

        # Create a unique key for this minute
        minute_key = (dt.year, dt.month, dt.day, dt.hour, dt.minute)

        if current_minute is None:
            current_minute = minute_key

        if minute_key == current_minute:
            buffer.append(price)
        else:
            # New minute has started, process the buffer
            if buffer:
                avg_price = sum(buffer) / len(buffer)
                year, month, day, hour, minute = current_minute

                row = [(year, month, day, hour, minute, avg_price)]
                columns = ["year", "month", "day", "hour", "minute", "price"]
                df = spark.createDataFrame(row, columns)

                df.write \
                    .mode("append") \
                    .partitionBy("year", "month", "day", "hour") \
                    .format("parquet") \
                    .save("hdfs:///bitcoin_stream_new")

                print(f"[{year}-{month:02d}-{day:02d} {hour:02d}:{minute:02d}] Avg price: {avg_price:.2f} written to HDFS.")

            # Reset for the new minute
            buffer = [price]
            current_minute = minute_key

except KeyboardInterrupt:
    pass
finally:
    if buffer:
        avg_price = sum(buffer) / len(buffer)
        year, month, day, hour, minute = current_minute

        row = [(year, month, day, hour, minute, avg_price)]
        columns = ["year", "month", "day", "hour", "minute", "price"]
        df = spark.createDataFrame(row, columns)

        df.write \
            .mode("append") \
            .partitionBy("year", "month", "day", "hour") \
            .format("parquet") \
            .save("hdfs:///bitcoin_stream_new")

        print(f"[{year}-{month:02d}-{day:02d} {hour:02d}:{minute:02d}] Final avg price written before shutdown.")

    consumer.close()
